In [1]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

from utils import seed_everything, set_device, get_train_val_dataloaders, train
from model import FeedForwardRegressionNet

from torch.utils.data import TensorDataset
from torch.optim import Adam

In [2]:
n_jobs = 1000
results = []

for i in range(n_jobs):
    results.append(pickle.load(open(f"results/training_stats_{i}", "rb")))

In [3]:
df = pd.DataFrame(results).sort_values("Valid. Loss")

In [4]:
df[0:10]

,Training Loss,Valid. Loss,Valid. CI,Training Time,Validation Time,hidden_dims,learning_rate,embed_dim0,embed_dim1,epochs,job_id
821,51089.387385,30865.464461,2619.610082,0:00:02,0:00:00,"[64, 128]",0.001,16,4,32,821
50,49506.401043,32587.331290,2670.961949,0:00:02,0:00:00,[64],0.030,4,4,32,50
870,64275.288451,32691.518901,3165.836053,0:00:02,0:00:00,[64],0.001,8,128,32,870
929,129907.132346,32961.374419,2731.566685,0:00:02,0:00:00,"[128, 16]",0.010,32,16,32,929
917,64752.878215,33009.842786,2787.259760,0:00:01,0:00:00,[64],0.030,32,8,4,917
815,111154.568709,33010.123566,2930.657364,0:00:03,0:00:00,"[64, 16]",0.003,8,128,16,815
279,75882.357491,34353.642009,2676.740552,0:00:02,0:00:00,"[64, 128]",0.030,128,8,16,279
453,59847.563249,35598.122540,2977.218850,0:00:03,0:00:00,[64],0.010,16,128,8,453
618,111552.135964,35700.833008,2857.035068,0:00:02,0:00:00,"[128, 16]",0.030,16,16,16,618
793,58642.629295,36226.115284,3108.037455,0:00:02,0:00:00,[64],0.010,32,64,8,793


In [5]:
best_hyperparameters = {
    "hidden_dims": [64, 128],
    "learning_rate": 0.01,
    "embed_dim0": 4,
    "embed_dim1": 8,
    "epochs": 32,
}

pickle.dump(best_hyperparameters, open("best_hyperparameters.pkl", "wb"))

In [6]:
hyperparameters = pickle.load(open("best_hyperparameters.pkl", "rb"))

HIDDEN_DIMS = hyperparameters["hidden_dims"]
LEARNING_RATE = hyperparameters["learning_rate"]
EMBED_DIM0 = hyperparameters["embed_dim0"]
EMBED_DIM1 = hyperparameters["embed_dim1"]
EPOCHS = hyperparameters["epochs"]

batch_size = 64
val_size = 0.1

In [7]:
device = set_device()

X = pickle.load(open("preprocessed_data/X_train_tensor.pkl", "rb"))
y = pickle.load(open("preprocessed_data/y_train_tensor.pkl", "rb"))

dataset = TensorDataset(X, y)

train_dataloader, validation_dataloader = get_train_val_dataloaders(
    dataset, val_size, batch_size
)

No GPU available, using the CPU instead.
45,000 training samples
4,999 validation samples


In [8]:
input_dim = X.shape[1] - 2
vocab_size0 = len(np.unique(X[:, 0]))
vocab_size1 = len(np.unique(X[:, 1]))
output_dim = 1

model = FeedForwardRegressionNet(
    input_dim,
    vocab_size0,
    vocab_size1,
    EMBED_DIM0,
    EMBED_DIM1,
    HIDDEN_DIMS,
    output_dim,
)

In [9]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

criterion = nn.MSELoss()

seed_everything(42)

training_stats = train(
    model,
    EPOCHS,
    train_dataloader,
    validation_dataloader,
    criterion,
    optimizer,
    device=device,
    SSY=y.var() * len(y),
)

torch.save(model.state_dict, "best_model.pt")


======== Epoch 1 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 247003.97
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 33885.38
  Validation SSE: 170614010.38
  Valida

  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 17024.80
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 17807.73
  Validation SSE: 89872833.46
  Validation took: 0:00:00
  Validation R^2: 0.9961900115013123

======== Epoch 10 / 32 ========
Trai

  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 10356.98
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 10467.07
  Validation SSE: 52795208.12
  Validation took: 0:00:00
  Validation R^2: 0.9977618

  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 7339.82
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 6480.07
  Validation SSE: 32731064.11
  Validation took: 0:00:00
  Validation R^2: 0.998612403